# Рекомендация тарифов

## Первичное ознакомление с данными

**ВСТУПЛЕНИЕ**  
Федеральный оператор сотовой связи "Мегалайн" предлагает своим клиентам два тарифных плана: "Смарт" и "Ультра". Однако, по данным компании многие клиенты пользуются устаревшими тарифами мобильного оператора.

В настоящий момент, "Мегалайн" планирует перевести клиентов, использующих архивные пакеты услуг, на новые тарифы, для чего требуется провеси анализ потребительского поведения клиентов. Необходимо создать систему, способную предлагать клиентам один из новых тарифов, который будет для них самым оптимальным, на основании особенностей их использования сотовой связи.  

**Цель исследования:** создание модели машинного обучения, решающей задачу классификации, которая предложит клиентам подходящий тариф.

In [32]:
# импортируем необходимые библиотеки:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [33]:
# отключаем предупреждения "Анаконды":
import warnings 
warnings.filterwarnings('ignore')

# увеличим дефолтный размер графиков:
from pylab import rcParams
rcParams['figure.figsize'] = 10,5

# пропишем путь к датасету:
data_cell = pd.read_csv('/datasets/users_behavior.csv')

#взглянем на таблицу с головы:
data_cell.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [34]:
#получим названия столбцов:
data_cell.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

In [35]:
#узнаем объём таблицы:
data_cell.shape

(3214, 5)

In [36]:
#получим информацию о количестве строк и форматах записи:
data_cell.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [37]:
# получим общее описания информации в столбцах:
data_cell.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Вывод:** информация о клиентах представлена в виде таблицы на 5 столбцах, в которой присутствует 3214 строк, иными словами в таблице присутствует 3214 объектов с 5 признаками. Каждая отдельная строка является отдельным наблюдением об одном клиенте: количество звонков в месяц, месячная продолжительность разговоров по телефону, количество СМС-сообщений, сумма затраченного трафика, а также информация об использываемом тарифе. Вся информация,  представлена в числовом формате: в столбце с тарифами в формате целого числа `int`, в остальных - в виде десятичной дроби `float`. 

Таким образом в нашем распоряжении обучающий набор данных и целевой признак, который нужно предсказать по остальным признакам. Такие задачи относятся к классу "Обучение с учителем". 

Поскольку целевой признак является категориальным, в нашем случае решается задача классификации. Так как категорий всего две, то здесь речь идет о бинарной (двоичной) классификации. 

## Разбейте данные на выборки

Для создания надежной модели машинного обучения, необходимо подготовить валидационную выборку. Данная выборка должна быть подготовлена до обучения модели. В противном случае, обучившись на тренировочном наборе, модель будет знать все ответы. Именно валидация покажет насколько правильно работает модель и нет ли в ней переобучения. 

Поскольку у нас нет скрытой тестовой выборки, значит, исходный датасет будет разбит на три части: обучающую, тестовую и валидационную. Обычно размеры валидационной и тестовой выборки равны. Размеры обучающей выборки к двум другим обычно находятся в соотношении 3:1:1.

В библиотеке `sklearn` для извления выборок имеется функция `train_test_split()`, аргументами которой передаются разделяемый датасет, размер валидационной выборки и определенный порядок случайностей в `random_state`.   

Сначала разделим общий датасет на обучающую и тестовую выборку, а затем полученную обучающую выборку опять разделим, но уже на обучающую и валидационную. Данный способ позволит избежать пересечения объектов во всех трех выборках.  

In [38]:
#разделим общий датасет "data_cell" на две выборки - обучающую и тестовую:
data_train, data_test = train_test_split(data_cell, test_size=0.2, random_state=54321)

#теперь разделим обучающий датасет "data_train" на обучающую и валидационную:
data_train, data_valid = train_test_split(data_train, test_size=0.25, random_state=54321)

#посмотрим на объёмы полученных выборок
print(data_train.shape)
print(data_test.shape)
print(data_valid.shape)

(1928, 5)
(643, 5)
(643, 5)


Объёмы выборок корректны. Теперь выделим в каждом датасете целевой признак, отделив его от остальных признаков, посредством методов `drop()` и `loc()`. 

In [39]:
#выделим признаки в обучающем наборе:
train_features = data_train.drop(['is_ultra'], axis=1)
train_target = data_train.loc[:, 'is_ultra']

# выделим признаки в тестовом наборе:
test_features = data_test.drop(['is_ultra'], axis=1)
test_target = data_test.loc[:, 'is_ultra']

# выделим признаки в валидационной выборке:
valid_features = data_valid.drop(['is_ultra'], axis=1)
valid_target = data_valid.loc[:, 'is_ultra']

**Вывод:** в связи с отсутствием тестовой выборки, исходный датасет был разделен на три части: обучающая выборка, тестовая и валидационная, путём использования метода `train_test_split()`. В каждой выборке целевой признак был отделен от остальных посредством использования двух методов `drop()` и `loc()`.

## Исследуйте модели

### Модель решающего дерева

Сначала обучим модель решающего дерева. Чтобы найти лучшие гиперпараметры, создадим цикл, который переберет настройки глубины и количество листьев. После это определим параметры модели, дающей лучший результат.

In [40]:
best_model = None
best_result = 0
for leaf in range(1, 6):
    for depth in range(1, 6):
        model = DecisionTreeClassifier(random_state = 54321, max_depth = depth, min_samples_leaf = leaf)
        model.fit(train_features, train_target)
        predictions = model.predict(valid_features)
        result = accuracy_score(valid_target, predictions)
        if result > best_result:
            best_model = model
            best_result = result

print('Accuracy лучшей модели:', best_result)
print(best_model)

Accuracy лучшей модели: 0.8195956454121306
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=54321, splitter='best')


Модель дерева решений дает точность ~82%, при максимальной глубине 5 уровней и минимальном количестве листьев в 5.

### Случайный лес

Теперь посмотрим какие результаты покажет случайный лес. Для этого также создадим цикл, который пройдется по количеству деревьев в самом лесу.

In [50]:
best_model = None
best_result = 0

for est in range(1, 11):
    for leaf in range(1, 6):
        for split in range(2, 6):
            model = RandomForestClassifier(random_state=54321, n_estimators=est, min_samples_leaf= leaf, min_samples_split = split) 
            model.fit(train_features, train_target) 
            predictions = model.predict(valid_features)
            result = accuracy_score(valid_target, predictions)
            if result > best_result:
                best_model = model
                best_result = result
     

print("Accuracy наилучшей модели на валидационной выборке:", best_result) 
print('Лучшая модель:', best_model)

Accuracy наилучшей модели на валидационной выборке: 0.8413685847589425
Лучшая модель: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=7,
                       n_jobs=None, oob_score=False, random_state=54321,
                       verbose=0, warm_start=False)


Случайный лес дает точность ~83%, чуть больше чем решающее дерево, при количестве 7 "судей" и 5 листьев

### Логистическая регресия

Для решения задачи классификации также подходит логистическая регрессия. Посмотрим, какой результат она покажет.

In [42]:
model = LogisticRegression(random_state=54321)
model.fit(train_features, train_target)
predictions = model.predict(valid_features)
result = accuracy_score(valid_target, predictions)

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7325038880248833


Логистическая регрессия показала низкий результат, поэтому для решения задачи она не подходит.   

**Вывод:** Лучшей моделью на валидационной выборке является модель случайного леса с количеством деревьев - 10

## Проверьте модель на тестовой выборке

In [43]:
#проверим лучшую модель дерева решения:
model = DecisionTreeClassifier(random_state=54321, max_depth = 5, min_samples_leaf=5)
model.fit(train_features, train_target)
predictions= model.predict(test_features)
result = accuracy_score(test_target, predictions)
print('Accuracy модели дерева решений на тестовой выборке:', result)

Accuracy модели дерева решений на тестовой выборке: 0.7713841368584758


In [51]:
#проверим лучшую модель случайного леса:
model = RandomForestClassifier(random_state=54321, n_estimators=7, min_samples_leaf = 5) 
model.fit(train_features, train_target) 
predictions = model.predict(test_features)
result = accuracy_score(test_target, predictions)
print('Accuracy модели случайного леса на тестовой выборке:', result)

Accuracy модели случайного леса на тестовой выборке: 0.7776049766718507


In [45]:
#также посмотрим и на логистическую регрессию:
model = LogisticRegression(random_state=54321)
model.fit(train_features, train_target)
predictions = model.predict(test_features)
result = accuracy_score(test_target, predictions)
print("Accuracy модели логистической регрессии на тестовой выборке:", result)

Accuracy модели логистической регрессии на тестовой выборке: 0.6780715396578538


**Вывод:** как мы можем видеть, у решающего дерева и случайного леса качество предсказаний упало, но остаётся на приемлемом уровне. Модель логистической регрессии на тестовой выборке не прошла проверку. Лучший результат показала модель случайного дерева.

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность, посмотрим какой класс самый встречаемый во всем датасете

In [46]:
data_cell['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Самым встречаемым классом является "0", то есть абоненты "Смарта". Создадим отдельный столбец, который будет состоять только из значений самого встречаемого класса, длина которого будет равно длине тестового, то есть создадим самую лучшую константную модель. 

In [47]:
const_preds_test = pd.Series(0, index=test_target.index) 
const_preds_test

1727    0
2885    0
2730    0
899     0
2119    0
       ..
1763    0
1323    0
2300    0
154     0
728     0
Length: 643, dtype: int64

Теперь сравним accuracy лчушей модели случайного леса и константной модели. 

In [48]:
model = RandomForestClassifier(random_state=54321, n_estimators=10) 
model.fit(train_features, train_target) 
model_preds_test = model.predict(test_features)
model_test_accuracy = accuracy_score(model_preds_test, test_target)
const_test_accuracy = accuracy_score(const_preds_test, test_target)
print(model_test_accuracy)
print(const_test_accuracy)
if model_test_accuracy > const_test_accuracy:
    print('Модель прошла проверку на адекватность')
else:
    print('Модель провалила проверку на адекватность')

0.7838258164852255
0.6640746500777605
Модель прошла проверку на адекватность


**Вывод:** модель прошла проверку на адекватность, поскольку дает результат лучше чем константная, то есть лучше, чем угадывание по самому распространенному классу. 

In [49]:
dummy_clf = DummyClassifier(strategy="prior")
dummy_clf.fit(train_features, train_target)
dummy_predict = dummy_clf.predict(test_features)
result = accuracy_score(test_target, dummy_predict)
print(result)

0.6640746500777605
